In [2]:
# imports
import sys
import numpy as np
import matplotlib.pyplot as plt
import emcee 
from time import time

## Task c: Reproduce Table III

We want to sample our parameter space for our features $\{a\}$, for different values of $k$ and $k_{max}$, using both our uniform and naturaleness priors as defined in the basic tasks. However, the main difference is that we need to marginalize out the extra parameters for higher values of $k_{max}$. 

In [33]:
# Define a modular model that generates features up to k_max: however we 
# then marginalize all larger k. 
def modular_model(a, x, kmax):
    '''
    Returns model of order kmax. a is a vector containing the model features.
    Note the kmax+1 in the for loop. This is due to range being from 0 to
    kmax-1.
    '''
    model = 0
    for k in range(kmax+1):
        model += a[k]*x**k
    return model



$\chi^2/\text{dof}$ means the value of $\chi^2$ per degree of freedom. 

Copy most of the code from the basic part.

In [34]:
# Load dataset
def load_data(file):
    d = {
        "x": [],
        "d": [],
        "sigma": []
    }
    # Skip first two rows, which are the header:
    with open(file) as f:
        for idx,line in enumerate(f):
            if idx < 3:
                pass
            else:
                val = line.split()
                d["x"].append(np.float(val[0]))
                d["d"].append(np.float(val[1]))
                d["sigma"].append(np.float(val[2]))
    # cast to numpy arrays
    d["x"] = np.array(d["x"])
    d["d"] = np.array(d["d"])
    d["sigma"] = np.array(d["sigma"])
    return d


file = 'D1_c_5.dat'
data = load_data(file)
display(data)

{'x': array([0.03183, 0.06366, 0.09549, 0.12732, 0.15915, 0.19099, 0.22282,
        0.25465, 0.28648, 0.31831]),
 'd': array([0.31694, 0.33844, 0.42142, 0.57709, 0.56218, 0.68851, 0.73625,
        0.8727 , 1.0015 , 1.0684 ]),
 'sigma': array([0.01585 , 0.01692 , 0.02107 , 0.02885 , 0.02811 , 0.03443 ,
        0.03681 , 0.04364 , 0.050075, 0.05342 ])}

In [35]:
def log_uniform_prior(a):
    '''
    Uniform prior, returns a log(1) if the values in a are in abs(a)<100. Note that this
    prior is not normalized. We take care of this later.
    '''
    if np.all(np.abs(a)<=100):
        return 0  # log(1)
    else: 
        return -np.inf  # log(0)
    

def log_naturaleness_prior(a, bar_a=5):
    '''Naturaleness prior implemented according to equation 24 with bar(a)=5. This ensures'''
    return -len(a)*np.log(np.sqrt(2*np.pi)*bar_a) - 1/2*(a.dot(a)/bar_a**2)

# Tests 
print(0 == log_uniform_prior([-1,2,50]))
print(-np.inf == log_uniform_prior([-1,2,500]))
print(-7.8651 == np.round(np.log((1/(np.sqrt(2*np.pi)*5))**3 * np.exp(-(1+4+9)/(2*5**2))),4) == np.round(log_naturaleness_prior(np.array([1,2,3]), 5),4))  # Calculate expression exact and log

True
True
True


In [36]:
def chi_squared(a, d, x, sigmas, kmax):
    '''
    Returns the chi squared measure for the datapoints d and x. The standard deviation is 
    assumed to be constant for all datapoints.
    '''
    chi_vec = (d-modular_model(a, x, kmax))/sigmas
    return np.sum(chi_vec**2)


def log_likelihood(a, d, x, sigmas, kmax):
    '''
    Returns log likelihood based on a Gaussian with di as the center values and 
    a standard deviation of sigma. a is the feature vector for our model.
    '''
    chi_sq = chi_squared(a, d, x, sigmas, kmax)
    like = -np.sum(np.log(np.sqrt(2*np.pi)*sigmas)) - 1/2*chi_sq
    return like


# Tests
a = np.array([1,2,3,4])
x = np.array([1,2])
d = np.array([2,4])
sigmas = np.array([2,4])
kmax = len(a)-1  # Sanity check that the model still works in the basic task case
print(f'Model: {[10, 49] == modular_model(a,x, kmax)}')
print(f'chi_squared: {142.5625==chi_squared(a,d,x,sigmas, kmax)}')
exact_likelihood = np.prod(1/(np.sqrt(2*np.pi)*sigmas)) * np.exp(-chi_squared(a,d,x,sigmas, kmax)/2)
print(f'log likelihood: {np.round(np.log(exact_likelihood),4) == np.round(log_likelihood(a, d, x, sigmas, kmax),4)}')  # exact calculation, and then log

Model: [ True  True]
chi_squared: True
log likelihood: True


#### Ok our new model passes this sanity check. All of our tests from the basic problem returns ok with the new modular model.

In [37]:
def log_post_uniform(a,d,x,sigma, kmax):
    return log_likelihood(a,d,x,sigma,kmax) + log_uniform_prior(a)


def log_post_natural(a,d,x,sigma, kmax, bar_a):
    return log_likelihood(a,d,x,sigma,kmax) + log_naturaleness_prior(a, bar_a=bar_a)

In [73]:
# Sampling

# Define constants and data
bar_a = 5
x = data["x"]
d = data["d"]
sigma = data["sigma"]
kmax = 5
k = min(kmax,2)  # k+1 is the numbers of relevant features, so our model is at most of degree 2.

# Define dimensions and walkers
ndim = kmax+1  # 0,1,...,kmax
nwalkers = ndim*2
# Initial guess
p0 = np.random.rand(ndim * nwalkers).reshape((nwalkers, ndim))


nburn = 200  # nbr of burning steps
nsamples = 1000  # nbr of final samples 

# additional arguments to our sampler: d, x, sigma and d,x,sigma, bar_a respectively
arglist_uniform = (d, x, sigma, kmax)
arglist_natural = (d, x, sigma, kmax, bar_a)

# Define samplers
sampler_uniform = emcee.EnsembleSampler(nwalkers, ndim, log_post_uniform, args=arglist_uniform)
sampler_natural = emcee.EnsembleSampler(nwalkers, ndim, log_post_natural, args=arglist_natural)
# Start sampler on posteriors. Use first few hundred iterations as burn in. 
t0 = time()  # start time
sampler_uniform.run_mcmc(p0, nburn + nsamples)
sampler_natural.run_mcmc(p0, nburn + nsamples)
t1 = time()  # end time
print(f'Sampling time: {t1-t0} seconds.')

Sampling time: 3.8116021156311035 seconds.


In [74]:
samples_uniform = sampler_uniform.chain[:,nburn:,:].reshape((-1,ndim))  # reshape to all samples per dim
sampler_natural = sampler_natural.chain[:,nburn:,:].reshape((-1,ndim)) 
print(samples_uniform.shape)

(12000, 6)


Now, marginalize out all features higher than order k=2, and compute the mean for our feature. We just do this by ignoring the other data corresponding to the higher features, since we are just interested in the number of times that we "land" on our relevant features anyway.


In [75]:
def present_feature_estimates(samples, prior_name, k, kmax):
    print(f'***** {prior_name} prior for kmax = {kmax} *****')
    a = np.zeros((k+1))
    for i in range(k+1):
        a[i] = samples[:,i].mean()
        print(f'avg(a_{i}) = {a[i]}')
    print('**************************************')
    return a

present_feature_estimates(samples_uniform, 'Uniform', k, kmax)
present_feature_estimates(sampler_natural, 'Gaussian (Natural)', k, kmax)
    
    

***** Uniform prior for kmax = 5 *****
avg(a_0) = 0.2755205448297961
avg(a_1) = 0.7459644247615889
avg(a_2) = 10.672640275150458
**************************************
***** Gaussian (Natural) prior for kmax = 5 *****
avg(a_0) = 0.2468305122930796
avg(a_1) = 1.6919339735926673
avg(a_2) = 2.6704201465702857
**************************************


array([0.24683051, 1.69193397, 2.67042015])